### Import dependencies

In [1]:
import tensorflow as tf
import os
import numpy as np
import pandas as pd
import pathlib

2023-05-10 17:50:28.844403: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-10 17:50:31.318557: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-10 17:50:31.319509: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-10 17:50:36.877382: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


### Import data

In [2]:
data_csv =  sorted(str(p) for p in pathlib.Path('data').glob("*.csv"))
data_csv

['data/aapl.csv',
 'data/abbv.csv',
 'data/abt.csv',
 'data/acn.csv',
 'data/adbe.csv',
 'data/amzn.csv',
 'data/avgo.csv',
 'data/bac.csv',
 'data/cmcsa.csv',
 'data/cost.csv',
 'data/crm.csv',
 'data/csco.csv',
 'data/cvx.csv',
 'data/dis.csv',
 'data/googl.csv',
 'data/hd.csv',
 'data/jnj.csv',
 'data/jpm.csv',
 'data/ko.csv',
 'data/lin.csv',
 'data/lly.csv',
 'data/ma.csv',
 'data/mcd.csv',
 'data/meta.csv',
 'data/mrk.csv',
 'data/msft.csv',
 'data/nke.csv',
 'data/nvda.csv',
 'data/pep.csv',
 'data/pfe.csv',
 'data/pg.csv',
 'data/tmo.csv',
 'data/tsla.csv',
 'data/unh.csv',
 'data/v.csv',
 'data/vz.csv',
 'data/wmt.csv',
 'data/xom.csv']

In [3]:
len(data_csv)

38

In [4]:
TrainX = []
TrainY = []
TestX = []
TestY = []
ValidX = []
ValidY = []

In [9]:
period = 20 # days
#TODO: Nomralization
for file in data_csv:
    ds = pd.read_csv(file)
    num_row = len(ds)
    num_train = int(num_row*0.70) - 20
    num_valid = int((num_row - num_train)/3)
    num_test = num_row - (num_train + num_valid)
    Y = ds['price']
    X = ds.iloc[:, 2:7]
    # print(X.head())
    for i in range(num_row-20):
        
        if i < num_train:
            TrainY.append(Y.iloc[i+period])
            print(type(X.iloc[i:i+period]))
            X_i = X.iloc[i:i+period].to_numpy()
            print(type(X_i))
            print(X_i)
            TrainX.append(X_i)
            break
        elif i < num_train + num_test:
            ValidY.append(Y.iloc[i+period])
            ValidX.append(X.iloc[i:i+period].to_numpy())
        else:
            TestY.append(Y.iloc[i+period])
            TestX.append(X.iloc[i:i+period].to_numpy())
    break

data/aapl.csv
5892
Unnamed: 0      2023-05-08
price                173.5
volume          55962793.0
rsi               67.42951
macd              2.897693
adx              33.451421
obv           4176464089.0
Name: 0, dtype: object
